## LOGO DETECTION PROJECT

Compulsory logos: Adidas, Nike, Puma, The North Face, Under Armour.

In [51]:
pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 27.1 MB/s eta 0:00:01
You should consider upgrading via the '/anaconda/envs/py38_default/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import csv
import cv2
import os

### DATA PRE-PROCESSING

In [3]:
data = []
labels = []
bboxes = []
imagePaths = []

In [12]:
base_dir = "images"
train_dir = os.path.join(base_dir, 'train')
noise_dir = os.path.join(base_dir, 'noise')
print('total training images:', len(os.listdir(train_dir)))
print('total noise images:', len(os.listdir(noise_dir)))

total training images: 40519
total noise images: 2979


In [5]:
data = pd.read_csv("annot_train.csv")
data.head()
print(len(data))

46163


In [5]:
file = "annot_train_scaled.csv"

with open(file, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    if header != None:
        for row in reader:
            labels.append(row[3])
            bboxes.append((np.float32(row[4]),np.float32(row[5]),np.float32(row[6]),np.float32(row[7])))  #ALREADY SCALED BETWEEN 0 AND 1
            imagePath = os.path.join(train_dir,row[0])
            imagePaths.append(imagePath)
            image = cv2.imread(imagePath)
            image = load_img(imagePath, target_size=(608, 608))
            image = img_to_array(image)
            data.append(image)

FileNotFoundError: [Errno 2] No such file or directory: 'DLCV_logo_project/train/london_1682860930385563294_20180101.jpg'

In [ ]:
file = "annot_noise_scaled.csv"

with open(file, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    if header != None:
        for row in reader:
            labels.append(row[3])
            bboxes.append((row[4],row[5],row[6],row[7]))  #ALREADY SCALED BETWEEN 0 AND 1
            imagePath = os.path.join(noise_dir,row[0])
            image = cv2.imread(imagePath)
            image = load_img(imagePath, target_size=(608, 608))
            image = img_to_array(image)
            data.append(image)

In [8]:
os.startfile(imagePaths[2])

In [9]:
data[0].shape

(608, 608, 3)

In [10]:
labels[50]

'Adidas'

In [11]:
bboxes[0]

(0.5296875, 0.615625, 0.5953125, 0.6859375)

In [12]:
len(labels),len(bboxes)

(150, 150)

In [13]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)

In [14]:
len(labels),len(bboxes)

(150, 150)

In [15]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [16]:
split = train_test_split(data, labels, bboxes, imagePaths,
test_size=0.2, random_state=42)

(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

In [17]:
len(trainBBoxes),len(testPaths)

(120, 30)

In [18]:
trainImages = np.array(trainImages)
testImages = np.array(testImages)
trainLabels = np.array(trainLabels)
testLabels = np.array(testLabels)
trainBBoxes = np.array(trainBBoxes)
testBBoxes = np.array(testBBoxes)
trainPaths = np.array(trainPaths)
testPaths = np.array(testPaths)

### MODEL PREPARATION

In [19]:
vgg = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(608, 608, 3)))

# freeze all VGG layers 
vgg.trainable = False

# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)

In [20]:
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
coordinates = Dense(4, activation="sigmoid",name="bounding_box")(bboxHead)

# construct a second fully-connected layer head, this one to predict
# the class label
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
probclass = Dense(5, activation="softmax",name="class_label")(softmaxHead)

# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model = Model(inputs=vgg.input,outputs=(coordinates, probclass))

In [21]:
# DEFINE IOU LOSS

@tf.function  
def IoUloss(y_true,y_pred):
    xA = max(y_true[0], y_pred[0])       
    yA = max(y_true[1], y_pred[1])
    xB = min(y_true[2], y_pred[2])
    yB = min(y_true[3], y_pred[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxtrueArea = (y_true[2] - y_true[0]) * (y_true[3] - y_true[1])    
    boxpredArea = (y_pred[2] - y_pred[0]) * (y_pred[3] - y_pred[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxtrueArea + boxpredArea - interArea)

    # return the intersection over union value
    return iou

In [22]:
IoUloss((0,0.2,0.4,0.6),(0.2,0,0.6,0.4))  #intersection over unit of two squares of 4x4 overlapping at the middle

<tf.Tensor: shape=(), dtype=float32, numpy=0.14285715>

In [39]:
g1 = tfa.losses.GIoULoss()

categorical_crossentropy= tf.keras.losses.CategoricalCrossentropy()

In [40]:
# define a dictionary to set the loss methods -- categorical
# cross-entropy for the class label head and mean absolute error
# for the bounding box head



losses = {
    "class_label": categorical_crossentropy,
    "bounding_box": g1,
}

# define a dictionary that specifies the weights per loss (both the
# class label and bounding box outputs will receive equal weight)
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

# initialize the optimizer, compile the model, and show the model
INIT_LR = 1e-4
NUM_EPOCHS = 20
BATCH_SIZE = 15

opt = Adam(learning_rate=INIT_LR)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 608, 608, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 608, 608, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 608, 608, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 304, 304, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [41]:
trainTargets = {"class_label": trainLabels,"bounding_box": trainBBoxes}
testTargets = {"class_label": testLabels,"bounding_box": testBBoxes}

In [42]:
print("[INFO] training model...")
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=1)

# serialize the model to disk
print("[INFO] saving object detector model...")
os.makedirs('models/') 
model.save("models/first.h5")

[INFO] training model...
Epoch 1/20
8/8 [==============================] - 108s 1s/step - loss: 5.4243 - bounding_box_loss: 1.1818 - class_label_loss: 4.2425 - bounding_box_accuracy: 0.6000 - class_label_accuracy: 0.1833 - val_loss: 2.8378 - val_bounding_box_loss: 0.9715 - val_class_label_loss: 1.8663 - val_bounding_box_accuracy: 0.5667 - val_class_label_accuracy: 0.2333
Epoch 2/20
8/8 [==============================] - 10s 1s/step - loss: 4.2368 - bounding_box_loss: 0.9861 - class_label_loss: 3.2508 - bounding_box_accuracy: 0.6000 - class_label_accuracy: 0.2583 - val_loss: 2.9340 - val_bounding_box_loss: 0.9719 - val_class_label_loss: 1.9621 - val_bounding_box_accuracy: 0.5667 - val_class_label_accuracy: 0.2000
Epoch 3/20
8/8 [==============================] - 10s 1s/step - loss: 3.8923 - bounding_box_loss: 0.9860 - class_label_loss: 2.9062 - bounding_box_accuracy: 0.6000 - class_label_accuracy: 0.2833 - val_loss: 2.8006 - val_bounding_box_loss: 0.9719 - val_class_label_loss: 1.8287 -